In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key = os.getenv('GROQ_API_KEY')
llm = ChatGroq(model_name='llama-3.1-70b-versatile',groq_api_key=groq_api_key)
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

d:\Langchain\venv2\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\Langchain\venv2\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [24]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
import bs4
loader = WebBaseLoader(
    web_paths=("https://docs.python.org/3/tutorial/controlflow.html",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs=loader.load()
docs

[Document(metadata={'source': 'https://lilianweng.github.io/faq/', 'title': "FAQ | Lil'Log", 'description': "FAQ - Lil'Log", 'language': 'en'}, page_content='\n\n\n\n\n\nFAQ | Lil\'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil\'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\nemojisearch.app\n\n\n\n\n\n\n\nFAQ\n\n\n\nQ: How can I get an update when a new post comes out?\nA: I post about my new post on this Twitter @lilianweng account. There is also a RSS feed.\n\n\nQ: What tool do you use for plotting?\nA: I\'m using Google Presentation (cloud version of PowerPoint).\n\n\nQ: What if I see something incorrect in the post?\nA: Send me an email: anotherlilian AT gmail.com if you have questions or find errors. Thanks \uf8ffüòä! But time of one person is limited and I\'m really busy nowadays, so there may take some time for me to get to process it.\n\n\nQ: Can I translate your pos

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [5]:
vector_store = Chroma.from_documents(embedding=embeddings,documents=splits)

d:\Langchain\venv2\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [6]:
retriver = vector_store.as_retriever()

In [7]:
## Prompt Template
system_prompt = (
    "you are AI assistant answer the question based on context"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
[
    ('system',system_prompt),
    ('human',"{input}")
]
)

In [8]:
## creating stufff document chain
question_answer_chain = create_stuff_documents_chain(llm,prompt)

## create retrival chain
rag_chain = create_retrieval_chain(retriver,question_answer_chain)

In [9]:
response = rag_chain.invoke({'input':'what is self-reflection?'})

In [10]:
response['answer']

"Self-reflection is a vital aspect that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It involves the agent reflecting on its own experiences, actions, and decisions to identify areas for improvement and learn from its mistakes. This self-reflection process enables the agent to refine its decision-making and problem-solving skills, leading to better performance over time.\n\nIn the context of the Reflexion framework, self-reflection is created by showing two-shot examples to a Large Language Model (LLM) and adding reflections into the agent's working memory. These reflections serve as context for querying the LLM and guide the agent's future behavior. Self-reflection is an essential component of the Reflexion framework, allowing the agent to learn from its experiences and adapt to new situations."

In [11]:
#### Adding the chat history

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

## prompt
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)


contextulize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ('system',contextualize_q_system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human','{input}'),
    ]
)



In [12]:
## create aware retriver

history_aware_retriver = create_history_aware_retriever(retriever=retriver,prompt=contextulize_q_prompt,llm=llm)

history_aware_retriver

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002B930D5CC70>))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate

In [13]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [14]:
## create stuff document chain
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)

## create retrival chain
rag_chain=create_retrieval_chain(history_aware_retriver,question_answer_chain)

In [15]:
from langchain_core.messages import AIMessage,HumanMessage

chat_history = []

question = 'What is self-reflection'

response1 = rag_chain.invoke({'input':question,'chat_history':chat_history})

chat_history.extend(
    [
    HumanMessage(content=question),
    AIMessage(content=response1['answer'])
    ]
)
question2 = 'Tell me more about it?'

response2 = rag_chain.invoke({'input':question2,'chat_history':chat_history})
response2['answer']

chat_history

[HumanMessage(content='What is self-reflection'),
 AIMessage(content="Self-reflection, in the context of autonomous agents, is the ability to analyze and evaluate their own past actions and decisions. It allows agents to identify mistakes, refine their decision-making processes, and improve their overall performance over time.\n\nIn the Reflexion framework, self-reflection is created by providing the agent with examples of failed trajectories and ideal reflections that guide future changes in the plan. These reflections are stored in the agent's working memory and used as context for querying the Language Model (LLM) to inform future decisions.\n\nSelf-reflection plays a crucial role in real-world tasks where trial and error are inevitable, enabling agents to learn from their mistakes and adapt to changing situations. It is distinct from the reflection mechanism, which synthesizes memories into higher-level inferences over time, guiding the agent's future behavior.\n\nIn essence, self-

In [16]:
## adding session_id

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def history_of_messages(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]

conversation_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    history_of_messages,
    input_messages_key='input',
    history_messages_key='chat_history',
    output_messages_key='answer'
    ) ## used for interactive chatbot with msg history




In [17]:
conversation_rag_chain.invoke(

    {"input":"What is task decompostion"},
    config={'configurable':{'session_id':'abcd'}},
)['answer']

'Task decomposition is a process of breaking down a complex task into smaller, simpler, and manageable sub-tasks or steps. This is done to make the task more understandable, easier to execute, and to facilitate problem-solving.\n\nIn the context of Large Language Models (LLMs) and autonomous agent systems, task decomposition is an essential component of planning. It involves taking a complicated task and decomposing it into a series of smaller tasks that can be executed by the agent.\n\nTask decomposition can be done in various ways, including:\n\n1. **Simple prompting**: Using simple prompts, such as "Steps for XYZ.\\n1.", to ask the LLM to break down a task into smaller steps.\n2. **Task-specific instructions**: Providing task-specific instructions, such as "Write a story outline." for writing a novel, to help the LLM decompose the task.\n3. **Human inputs**: Using human inputs, such as providing a list of sub-tasks or steps, to help the LLM decompose the task.\n4. **Chain of thought

In [20]:
conversation_rag_chain.invoke(

    {"input":"What are the types of memory?"},
    config={'configurable':{'session_id':'abcd'}},
)['answer']

'You previously asked about the types of memory. According to the text, there are several types of memory in human brains:\n\n1. **Sensory Memory**: This is the earliest stage of memory, providing the ability to retain impressions of sensory information (visual, auditory, etc) after the original stimuli have ended. Sensory memory typically only lasts for up to a few seconds. Subcategories include:\n\t* **Iconic memory** (visual)\n\t* **Echoic memory** (auditory)\n\t* **Haptic memory** (touch)\n2. **Short-Term Memory (STM) or Working Memory**: This stores information that we are currently aware of and needed to carry out complex cognitive tasks such as learning and reasoning. Short-term memory is believed to have the capacity of about 7 items and lasts for 20-30 seconds.\n3. **Long-Term Memory (LTM)**: Long-term memory can store information for a remarkably long time, ranging from a few days to decades, with an essentially unlimited storage capacity. There are two subtypes of LTM:\n\t* 

In [19]:
store

{'abcd': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is task decompostion'), AIMessage(content='Task decomposition is a process of breaking down a complex task into smaller, simpler, and manageable sub-tasks or steps. This is done to make the task more understandable, easier to execute, and to facilitate problem-solving.\n\nIn the context of Large Language Models (LLMs) and autonomous agent systems, task decomposition is an essential component of planning. It involves taking a complicated task and decomposing it into a series of smaller tasks that can be executed by the agent.\n\nTask decomposition can be done in various ways, including:\n\n1. **Simple prompting**: Using simple prompts, such as "Steps for XYZ.\\n1.", to ask the LLM to break down a task into smaller steps.\n2. **Task-specific instructions**: Providing task-specific instructions, such as "Write a story outline." for writing a novel, to help the LLM decompose the task.\n3. **Human inputs**: Using human